## Toronto Neighbourhoods

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

### Parsing a wikipedia table with BeautifulSoup to obtain table values

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})

tdlist = table.findAll('td')

In [3]:
# first element of tdlist is the postcode, second element is the borough name and third element is the neighbourhood name

postcodes = []
bor = []
neigh = []

count = 1
for idx, l in enumerate(tdlist):    
    if count == 1:
        postcodes.append(l.get_text())
        count += 1
    elif count == 2:
        bor.append(l.get_text())
        count += 1
    elif count == 3:
        neigh.append(l.get_text().strip())
        count = 1


### Constructing a dataframe with the Toronto neighbourhood information

In [5]:
toronto = pd.DataFrame()
toronto['Postcode'] = postcodes
toronto['Borough'] = bor
toronto['Neighbourhood'] = neigh

In [6]:
# remove rows where the borough is 'Not assigned' to a  postcode

toronto = toronto[toronto['Borough'] != 'Not assigned']

In [7]:
# search for not assigned neighbourhoods and name them after the borough

for idx, row in toronto.iterrows():
    if str(row['Neighbourhood']) == 'Not assigned':
        toronto.at[idx, 'Neighbourhood'] = str(row['Borough'])

In [8]:
# group neighbourhoods with the same postcode

toronto = toronto.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [11]:
toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
toronto.shape 

(103, 3)